In [1]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import os

## Save best training metrics

In [20]:
with open(r'Z:\grodriguez\CardiacOCT\predicted_results\summary.json') as f:
    summary = json.load(f)

In [25]:
#Merge frames into pullbacks

pullbacks_origs = os.listdir(r'Z:\grodriguez\CardiacOCT\predicted_results')
pullbacks_origs_set = []
pullbacks_dict = {}

for i in range(len(pullbacks_origs)):
    if pullbacks_origs[i].split('_frame')[0] not in pullbacks_origs_set:
        pullbacks_origs_set.append(pullbacks_origs[i].split('_frame')[0])

    else:
        continue

for i in range(len(pullbacks_origs_set)):
    frames_from_pullback = [frame for frame in pullbacks_origs if pullbacks_origs_set[i] in frame]
    pullbacks_dict[pullbacks_origs_set[i]] = frames_from_pullback

#Remove last key-value pair
keys = list(pullbacks_dict.keys())[-3:]
for key in keys:
    pullbacks_dict[key].pop()
    if not pullbacks_dict[key]:
        pullbacks_dict.pop(key)

In [27]:
pullbacks_dict['NLDAMPH0005_1']

['NLDAMPH0005_1_frame0_003.nii.gz',
 'NLDAMPH0005_1_frame120_003.nii.gz',
 'NLDAMPH0005_1_frame160_003.nii.gz',
 'NLDAMPH0005_1_frame200_003.nii.gz',
 'NLDAMPH0005_1_frame240_003.nii.gz',
 'NLDAMPH0005_1_frame268_003.nii.gz',
 'NLDAMPH0005_1_frame280_003.nii.gz',
 'NLDAMPH0005_1_frame320_003.nii.gz',
 'NLDAMPH0005_1_frame360_003.nii.gz',
 'NLDAMPH0005_1_frame400_003.nii.gz',
 'NLDAMPH0005_1_frame40_003.nii.gz',
 'NLDAMPH0005_1_frame440_003.nii.gz',
 'NLDAMPH0005_1_frame449_003.nii.gz',
 'NLDAMPH0005_1_frame480_003.nii.gz',
 'NLDAMPH0005_1_frame520_003.nii.gz',
 'NLDAMPH0005_1_frame80_003.nii.gz']

In [28]:
def merge_dicts(list_dicts):

    result = {}
    for d in list_dicts:
        for label, metrics in d.items():
            if label not in result:
                if np.isnan(metrics['Dice']):
                    metrics['Dice'] = 0.0
                result[label] = [metrics['Dice']]

            else:
                if np.isnan(metrics['Dice']):
                    metrics['Dice'] = 0.0
                result[label].append(metrics['Dice'])
            
    return result
    

In [29]:
def mean_metrics(dict_pullback):
    
    for label, dices in dict_pullback.items():
        dict_pullback[label] = np.mean(dices)

    return dict_pullback

In [8]:
list_dicts_pullback = []
final_dict = {}

for pullback in pullbacks_dict:

    for frame in pullbacks_dict[pullback]:

        for sub_dict in summary['results']['all']:

            if sub_dict['test'] == r'/mnt/netcache/diag/grodriguez/CardiacOCT/predicted_results/{}'.format(frame):
                list_dicts_pullback.append({k: v for i, (k, v) in enumerate(sub_dict.items()) if i < len(sub_dict) - 2})

            else:
                continue

    result = merge_dicts(list_dicts_pullback)
    mean_result = mean_metrics(result)
    final_dict[pullback] = mean_result

In [15]:
summary['results']['all'][0].keys()

dict_keys(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9', 'reference', 'test'])

In [16]:
final_dict.keys()

dict_keys(['NLDAMPH0067_1', 'NLDAMPH0063_1', 'NLDHMC0009_1', 'NLDRADB0097_1', 'NLDRADB0094_1', 'NLDAMPH0007_1', 'NLDISALA0090_1', 'NLDRADB0085_1', 'NLDAMPH0076_1', 'NLDRADB0088_1', 'NLDAMPH0029_1', 'NLDAMPH0003_1', 'NLDISALA0008_1', 'NLDAMPH0069_1', 'NLDRADB0095_1', 'NLDRADB0084_2', 'NLDAMPH0012_1', 'NLDAMPH0066_1', 'NLDAMPH0059_1', 'NLDAMPH0061_1', 'ESTNEMC0027_1', 'NLDRADB0090_1', 'NLDISALA0085_1', 'NLDISALA0092_1', 'NLDAMPH0017_1', 'NLDHMC0009_2', 'NLDISALA0095_2', 'NLDRADB0094_2', 'NLDAMPH0045_1', 'NLDRADB0086_1', 'NLDISALA0081_1', 'NLDAMPH0073_1', 'NLDAMPH0013_1', 'NLDISALA0079_1', 'NLDRADB0084_1', 'NLDISALA0086_1', 'NLDISALA0081_2', 'NLDAMPH0075_1', 'NLDRADB0091_1', 'NLDISALA0006_1', 'ESTNEMC0027_2', 'NLDISALA0084_1', 'NLDHMC0002_1', 'NLDAMPH0068_1', 'NLDISALA0082_1', 'NLDAMPH0072_1', 'NLDRADB0096_1', 'NLDISALA0088_1', 'NLDHMC0002_2', 'NLDISALA0058_1', 'NLDISALA0076_1', 'NLDAMPH0065_1', 'NLDISALA0070_1', 'NLDISALA0095_1', 'NLDISALA0096_1', 'NLDAMPH0062_1'])

In [95]:
with open(r'Z:\grodriguez\CardiacOCT\metrics_build\pullback_metrics.json', 'w') as f:
    json.dump(final_dict, f, indent=4)

## Create Excel file

### Second sheet (best performance on 1st dataset)

In [17]:
with open(r'Z:\grodriguez\CardiacOCT\metrics_build\pullback_metrics_model1.json', 'r') as f:
    data = pd.read_json(f, orient='index')

In [18]:
data.rename(columns={
        1: "lumen",
        10: "wthrombus",
        11: "dissection",
        12: "rupture",
        2: "guidewire",
        3: "wall",
        4: "lipid",
        5: "calcium",
        6: "media",
        7: "catheter",
        8: "sidebranch",
        9: "rthrombus"}, inplace=True)

In [19]:
#CHANGE NAME EACH TIME IS COMPILED
data.to_excel('./metrics_models.xlsx')

In [3]:
a = [np.nan, np.nan, np.nan]

np.nanmean(a)

C:\Users\gonza\AppData\Local\Temp\ipykernel_3424\351886512.py:3: RuntimeWarning: Mean of empty slice
  np.nanmean(a)


nan